In [1]:
# Torch
import torch
import torch.utils.data as data
from torchsummary import summary

# Numpy, scipy, scikit-image, spectral
import numpy as np
import sklearn.svm
import sklearn.model_selection
from skimage import io
# Visualization
import seaborn as sns
import visdom

import pickle
import scipy.io

import os
from utils import metrics, convert_to_color_, convert_from_color_,\
    display_dataset, display_predictions, explore_spectrums, plot_spectrums,\
    sample_gt, build_dataset, show_results, compute_imf_weights, get_device
from datasets import get_dataset, HyperX, open_file, DATASETS_CONFIG
from models import get_model, train, test, save_model

print(torch.__version__)
print(f'Cuda available? {torch.cuda.is_available()}')

1.11.0+cu102
Cuda available? True


In [2]:
# Parameters
DATASET = 'IndianPines'
MODEL = 'hamida'
PARAMS = './checkpoints/hamida_et_al/IndianPines/params.pkl'
CHECKPOINT = './checkpoints/hamida_et_al/IndianPines/run_epoch50_0.95.pth'

In [12]:
# Get data
img, gt, label_values, ignored_labels, rgb_bands, palette = get_dataset(DATASET,target_folder='./Datasets/')
print(img.shape)
print(gt.shape)
print()

# Subsample
sub_amt = 3
sub_dir = f'./Datasets/IndianPines{sub_amt}/'
sub_img = img[:,:,::sub_amt].copy()
print(sub_img.shape)
img_dict = {}
img_dict[f'indian_pines_{sub_amt}'] = sub_img
if not os.path.isdir(sub_dir):
    os.mkdir(sub_dir)
scipy.io.savemat(f'{sub_dir}Indian_pines_{sub_amt}.mat', img_dict)

(145, 145, 200)
(145, 145)

(145, 145, 67)


In [3]:
# Load subsampled image
img, gt, label_values, ignored_labels, rgb_bands, palette = get_dataset('IndianPines3',target_folder='./Datasets/')
print(img.shape)

(145, 145, 67)


In [9]:
# Upload parameters
with open(PARAMS, 'rb') as handle:
    hyperparams = pickle.load(handle)
    
# Load model
model, _, _, hyperparams = get_model(MODEL, **hyperparams)
model.load_state_dict(torch.load(CHECKPOINT))
probabilities = test(model, img, hyperparams)
prediction = np.argmax(probabilities, axis=-1)
print(prediction.shape)

Inference on the image: 100%|████████████████| 196/196 [00:00<00:00, 500.28it/s]

(145, 145)


In [34]:
from os import listdir
from os.path import isfile, join

onlyfiles = [f for f in listdir('./checkpoints/hamida_et_al/IndianPines/') if isfile(join('./checkpoints/hamida_et_al/IndianPines/', f))]
print(onlyfiles)
print()
accs = np.array([float(epoch_name[len(epoch_name)-8:len(epoch_name)-4]) for epoch_name in onlyfiles])
print(accs)
print()
sorted_accs = np.argsort(accs)
print(onlyfiles[sorted_accs[-1]])

['run_epoch19_0.79.pth', 'run_epoch4_0.49.pth', 'run_epoch12_0.81.pth', 'run_epoch20_0.76.pth', 'run_epoch16_0.66.pth', 'run_epoch6_0.53.pth', 'run_epoch15_0.86.pth', 'run_epoch5_0.49.pth', 'run_epoch10_0.61.pth', 'run_epoch18_0.79.pth', 'run_epoch17_0.79.pth', 'run_epoch9_0.55.pth', 'run_epoch17_0.66.pth', 'run_epoch5_0.56.pth', 'run_epoch8_0.68.pth', 'run_epoch6_0.60.pth', 'run_epoch11_0.71.pth', 'run_epoch18_0.65.pth', 'run_epoch2_0.43.pth', 'run_epoch8_0.59.pth', 'run_epoch7_0.57.pth', 'run_epoch19_0.63.pth', 'run_epoch10_0.73.pth', 'run_epoch1_0.40.pth', 'run_epoch14_0.68.pth', 'run_epoch11_0.63.pth', 'run_epoch20_0.86.pth', 'run_epoch3_0.54.pth', 'run_epoch2_0.52.pth', 'run_epoch15_0.68.pth', 'run_epoch1_0.41.pth', 'run_epoch7_0.64.pth', 'run_epoch12_0.42.pth', 'run_epoch16_0.86.pth', 'run_epoch3_0.44.pth', 'run_epoch13_0.60.pth', 'run_epoch13_0.69.pth', 'run_epoch14_0.78.pth', 'run_epoch9_0.74.pth']

[0.79 0.49 0.81 0.76 0.66 0.53 0.86 0.49 0.61 0.79 0.79 0.55 0.66 0.56
 0.68 0.

In [23]:
epoch1 = onlyfiles[0]
print(len(epoch1))
converted = float(epoch1[12:16])
print(type(converted))

20
<class 'float'>
